In [2]:
import pandas as pd
import re
# df = pd.read_csv(r'C:\Users\zqy\Desktop\新建文件夹\厦门POC数据\火眼模板\0402\enc_交易流水-图空间1-图空间2-模型2.csv',engine = 'python',dtype = 'object')
df = pd.read_excel(r'C:\Users\zqy\Desktop\新建文件夹\厦门POC数据\火眼模板\0402\流水待脱敏.xlsx',encoding ='gbk')
print(df.dtypes)
df = df.applymap(lambda x: re.sub('nan','',str(x)))

银行            float64
客户名称           object
客户证件号码         object
查询账号           object
查询卡号           object
交易对方证件号码       object
交易对方账号        float64
交易对方卡号         object
交易对方名称         object
交易类型           object
借贷标志           object
币种             object
交易金额          float64
交易余额          float64
交易时间           object
交易流水号          object
交易对方余额        float64
交易对方账号开户行      object
交易摘要           object
交易网点名称         object
交易网点代码        float64
日志号            object
传票号            object
凭证种类           object
凭证号            object
现金标志           object
终端号            object
交易是否成功        float64
交易发生地          object
商户名称          float64
商户号           float64
本方IP地址         object
本方MAC地址        object
对方IP地址         object
对方MAC地址        object
交易柜员号          object
交易渠道          float64
备注             object
对公对私标志          int64
交易对手对公对私标志      int64
dtype: object


In [6]:
#对8位及以下、30位及以上、包含英文字母，不是银行卡的交易对方卡号，置空处理
df_对手 = df.groupby(['交易对方卡号','交易对方名称']).size().reset_index()
df_异常对手 = df_对手[df_对手['交易对方卡号'].map(lambda x: len(x)<=8 or len(x)>=30 or len(re.findall(r'([a-zA-Z]+)',x))>0 )]
df['交易对方卡号'] = df.apply(lambda x: '' if x['交易对方卡号'] in df_异常对手['交易对方卡号'].tolist() else x['交易对方卡号'],axis = 1)
print(df[df['交易对方卡号']==''].shape)
print(df_对手)

       银行          客户名称              客户证件号码                 查询账号  \
37          南京健宇达建材有限公司  91320111093938726W    01600120210017464   
205         南京健宇达建材有限公司  91320111093938726W    01600120210017464   
274         南京健宇达建材有限公司  91320111093938726W    01600120210017464   
316         南京健宇达建材有限公司  91320111093938726W    01600120210017464   
336         南京健宇达建材有限公司  91320111093938726W    01600120210017464   
358         南京健宇达建材有限公司  91320111093938726W    01600120210017464   
598         南京峰宁江建材有限公司           067085416    01690120210004417   
942         南京峰宁江建材有限公司           067085416    01690120210004417   
1140        南京峰宁江建材有限公司           067085416    01690120210004417   
1160        南京峰宁江建材有限公司           067085416    01690120210004417   
1204        南京峰宁江建材有限公司           067085416    01690120210004417   
1304        南京峰宁江建材有限公司           067085416    01690120210004417   
1363        南京峰宁江建材有限公司           067085416    01690120210004417   
1403        南京峰宁江建材有限公司           067085416    0

In [7]:
# 提取交易对方卡号中收单机构，提取内容输出到df_匹配
df_交易对手 = df_对手
df_匹配 = pd.DataFrame()
df_收单代码 = pd.read_excel(r'C:\Users\zqy\Documents\火模\示例模型\示例数据\收单机构.xlsx',encoding = 'gbk',dtype = object)
for str_regex in df_收单代码['正则匹配'].tolist():
    df_筛选 = df_交易对手[df_交易对手['交易对方卡号'].str.contains(str_regex)]
    df_匹配 = df_匹配.append(df_筛选)
    print(str_regex)
    print(df_筛选)
    

^801[0-9]{12}$
                交易对方卡号             交易对方名称  0
40555  801000006816188  邳州市建设工程招标投标市场管理中心  7
^802[0-9]{12}$
                交易对方卡号               交易对方名称    0
40585  802000000961088       张家港市佑达建设工程有限公司   14
40586  802000000961088        水立通建设(江苏)有限公司    4
40587  802000000961088        水立通建设（江苏）有限公司   14
40588  802000014535788     张家港市锦丰城乡开发投资有限公司    2
40589  802000014679188    江苏扬子江国际冶金工业园建设管理处    8
40590  802000031960588   江苏齐力建设集团齐赢路面工程有限公司    2
40591  802000039838388        张家港台恒电子衡器有限公司    2
40592  802000050305088         张家港市喜鼎贸易有限公司    1
40593  802000050592288         张家港市鼎臻贸易有限公司    2
40594  802000051083188      江苏金沙洲旅游投资发展有限公司    1
40595  802000056891188      江苏金沙洲旅游投资发展有限公司    1
40596  802000060775188          江苏达尔瑞贸易有限公司    9
40597  802000060908888       张家港科贝奇机械科技有限公司    1
40598  802000063517588           南通提图商贸有限公司    1
40599  802000065627988           长顺建设集团有限公司  160
40600  802000066005888           昆山沪顺建材有限公司    8
40601  802000066028988         江苏港顺企业发展有限公司    

^834[0-9]{12}$
                交易对方卡号               交易对方名称    0
41020  834690707420000       现代金融控股(成都)有限公司  138
41021  834690707420000       现代金融控股（成都）有限公司  117
41022  834690707420000   现代金融控股（成都）有限公司重庆分公  113
41023  834690707420000  现代金融控股（成都）有限公司重庆分公司   33
41024  834690707420000     （特约）现代金控重庆（清结算2）    1
^835[0-9]{12}$
                交易对方卡号              交易对方名称  0
41025  835440145110032  （特约）北京华胜平航空旅游公司(互联  1
^836[0-9]{12}$
                交易对方卡号     交易对方名称   0
41028  836100048990015       4830  13
41029  836100048990015       中国银联   9
41035  836301057222135  南京市沐潼家电销售   1
^837[0-9]{12}$
                交易对方卡号            交易对方名称  0
41038  837110060125130  （特约）中信百信银行股份有限公司  1
^838[0-9]{12}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^839[0-9]{12}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^841[0-9]{12}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^842[0-9]{12}$
                交易对方卡号           交易对方名称    0
41057  842440359330001   （特约）腾讯（理财通/还款）    1

^303[0-9]{12}$
                交易对方卡号            交易对方名称  0
13477  303000094980001     （特约）光大银行信用卡还款  9
13478  303102289990010  中国光大银行股份有限公司北京分行  4
13479  303102289990010   分行特色业务（满帮代付专用户）  1
13480  303102289990010              南京分行  1
13482  303980094985841  光大卡中心信用卡还款（深圳线上）  7
^304[0-9]{12}$
                交易对方卡号          交易对方名称  0
13484  304101040013624    江西网优科技股份有限公司  1
13485  304102160120019     （特约）爱钱进（提现）  1
13486  304143065130005      天成物业服务有限公司  1
13487  304301053118090     南京市六合区忠辉烟酒店  1
13488  304335059982069    海宁市海洲秋水衣人皮草行  1
13489  304350248160001  华夏银行股份有限公司厦门分行  2
13490  304350248160003     拉卡拉支付股份有限公司  2
13491  304350260110005  华夏银行股份有限公司厦门分行  1
^305[0-9]{12}$
                交易对方卡号             交易对方名称   0
13541  305290150450006       中国民生银行代付（上海）  85
13542  305393048160002   中国民生银行股份有限公司厦门分行   8
13543  305393048160002               中国银联   1
13544  305393148160006       上海汇付数据服务有限公司   2
13545  305393148160030       浙江网商银行股份有限公司   1
13546  305420180623002  华中科技大学同济医学院附属

^4145210[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4145310[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4154560[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4162640[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^417[0-9]{12}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4171100[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4172210[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4184930[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4185050[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4192310[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4202220[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4221200[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4233310[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4243010[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易

^5197117[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5207020[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5213000[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5238330[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5238333[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5247410[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5253450[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5264330[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5274550[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5284630[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5295280[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5303380[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5311930[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5326550[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号,

^4486400[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4498500[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4505800[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4511900[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4526500[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4538200[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4542200[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4551600[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4551601[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4551610[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4561100[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4572600[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4588500[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^4595210[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号,

^5585511[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5585514[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5585525[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5585541[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5585586[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5585591[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5585623[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5585624[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5585635[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5587317[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5587432[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5587433[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5587580[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
^5805364[0-9]{8}$
Empty DataFrame
Columns: [交易对方卡号,

In [8]:
df['交易对方卡号'] = df.apply(lambda x: (x['查询卡号']+'_'+x['交易对方卡号']) if x['交易对方卡号'] in df_匹配['交易对方卡号'].tolist() else x['交易对方卡号'],axis = 1)
print(df[df['交易对方卡号'].str.contains('802000000961088')])

      银行        客户名称      客户证件号码                  查询账号                  查询卡号  \
61290     长顺建设集团有限公司  74941470-7  32201986444051501547  32201986444051501547   
61466     长顺建设集团有限公司  74941470-7  32201986444051501547  32201986444051501547   
61682     长顺建设集团有限公司  74941470-7  32201986444051501547  32201986444051501547   
61732     长顺建设集团有限公司  74941470-7  32201986444051501547  32201986444051501547   
61877     长顺建设集团有限公司  74941470-7  32201986444051501547  32201986444051501547   
61915     长顺建设集团有限公司  74941470-7  32201986444051501547  32201986444051501547   
64354     长顺建设集团有限公司  74941470-7  32201986444051501547  32201986444051501547   
64752     长顺建设集团有限公司  74941470-7  32201986444051501547  32201986444051501547   
65236     长顺建设集团有限公司  74941470-7  32201986444051501547  32201986444051501547   
66286     长顺建设集团有限公司  74941470-7  32201986444051501547  32201986444051501547   
67675     长顺建设集团有限公司  74941470-7  32201986444051501547  32201986444051501547   
67684     长顺建设集团有限公司  74941470-7  322019

In [9]:
# 提取交易对方卡号中三方，提取内容输出到df_匹配2
df_交易对手2 = df_交易对手[~df_交易对手['交易对方卡号'].isin(df_匹配['交易对方卡号'])]
df_匹配2 = pd.DataFrame()
df_筛选2 = df_交易对手2[df_交易对手2['交易对方卡号'].str.contains('^2[0-9\*]{8}$')]
df_匹配2 = df_匹配2.append(df_筛选2)
print(df_筛选2)


df_挂牌三方 = pd.read_excel(r'C:\Users\zqy\Documents\火模\示例模型\示例数据\中国人民银行挂牌第三方支付机构.xlsx',encoding = 'gbk',dtype = object)
for str_regex in df_挂牌三方['简称']:
    df_筛选 = df_交易对手[df_交易对手['交易对方名称'].str.contains(str_regex)]
    df_匹配2 = df_匹配2.append(df_筛选)
    print(str_regex)
    print(df_筛选)

df_交易对手3 = df_交易对手2[~df_交易对手2['交易对方卡号'].isin(df_匹配2['交易对方卡号'])]
print(df_交易对手3.shape)

          交易对方卡号                   交易对方名称     0
12918  210401293                             45
12919  210401293           网银在线（北京）科技有限公司    18
12920  210401294                 易宝支付有限公司     3
12921  210401324              拉卡拉支付股份有限公司     1
12922  210401325                 中金支付有限公司    39
12923  210401328              北京百付宝科技有限公司    43
12924  210401344                             37
12925  210401344            北京钱袋宝支付技术有限公司    15
12926  210401366                随行付支付有限公司     4
12927  210401378              开联通支付服务有限公司     2
12928  210401381            易智付科技（北京）有限公司     8
12942  211800293            金运通网络支付股份有限公司     4
12945  212100124             武汉合众易宝科技有限公司     3
12957  215500684             快钱支付清算信息有限公司    10
12958  215500685               通联支付(商户结算)     1
12959  215500685           通联支付网络服务股份有限公司    10
12960  215500687             上海汇付数据服务有限公司    12
12961  215500688           上海富友支付服务股份有限公司     4
12962  215500689           宝付网络科技（上海）有限公司     9
12963  215500690                        

财付通
                            交易对方卡号             交易对方名称     0
398                                               财付通    23
399                                          财付通-QQ转账     3
400                                      财付通-七欣天南京十一店     1
401                                    财付通-世界茶饮南京溧水万达     1
402                                          财付通-东升饭店     1
403                                         财付通-个体户王娟     1
404                                    财付通-中国人民财产保险股份     1
405                                      财付通-中国石化江苏石油     6
406                                    财付通-中国铁路总公司资金清    13
407                                    财付通-中油溧水石油化工有限     2
408                                          财付通-丰巢科技     2
409                                       财付通-乐享生活超市店     1
410                                          财付通-乐基广场     1
411                                         财付通-九江优壹朵     1
412                                    财付通-九江新天地建国酒店有     2
413                                 

快付通
                交易对方卡号              交易对方名称   0
13141        243300146  深圳市快付通金融网络科技服务有限公司   2
40951  824440359330023             （特约）快付通  14
40953  824440360510056             （特约）快付通  13
广州银联
                     交易对方卡号             交易对方名称   0
19576   3602008119200030379  广州银联网络支付有限公司客户备付金  18
19579   3602008129200787189  广州银联网络支付有限公司客户备付金   8
19580   3602008129200787216  广州银联网络支付有限公司客户备付金   7
21767  44001400101053006593  广州银联网络支付有限公司客户备付金   2
21769  44001401009053001857  广州银联网络支付有限公司客户备付金   1
数字王府井
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
银联商务
                    交易对方卡号           交易对方名称    0
3275   1001281229006644228    银联商务有限公司客户备付金   47
19577  3602008119200244143    银联商务有限公司客户备付金  191
19578  3602008119200244143  银联商务股份有限公司客户备付金    3
21756         437765507839    银联商务有限公司客户备付金    1
裕福支付
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
易生
                 交易对方卡号       交易对方名称  0
1758   0000300000000264  易生支付有限公司备付金  1
41981      991110000199  易生支付有限公司备付金  1
银盛


易票联
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
深银联易
                     交易对方卡号       交易对方名称  0
21859  44201517600052540329  深圳市深银联易办事公司  1
银生宝
                交易对方卡号              交易对方名称  0
41172  876310042140001  （特约）绥芬河通达(银生宝-交通物流  3
银联金融
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
宝付
                     交易对方卡号                         交易对方名称   0
119                                         宝付网络科技（上海）有限公司   3
7965        105551000020704                          宝付支付-   5
7966        105551000020704                 宝付网络科技（上海）有限公司  16
7967        105551000020704  宝付网络科技（上海）有限公司-上海岱海金融信息服务有限公司   1
7968        105551000020704    宝付网络科技（上海）有限公司-晋商消费金融股份有限公司   2
7969        105551000020704      宝付网络科技（上海）有限公司-深圳众联商务有限公司   5
7970        105551000020704        宝付网络科技（上海）有限公司-维仕担保有限公司   4
8794         11013545952701            宝付网络科技（上海）有限公司客户备付金   2
12962             215500689                 宝付网络科技（上海）有限公司   9
19884   4000040429200135339            宝付网络科技（上海）有限公司客户备付金 

易通商联
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
西邮寄
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
恒信通
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
和融通
               交易对方卡号    交易对方名称  0
8318  105584000000016  财付通-三和融通  1
商银信
                          交易对方卡号               交易对方名称  0
5294             105110073991392  商银信支付服务有限责任公司-爱达屋酒店  1
8730   11001029300053001700-0002   商银信支付服务有限责任公司客户备付金  4
14056            317113148163019       （特约）商银信（商户清结算）  2
14060            317113148164019       （特约）商银信（商户清结算）  5
19901            401110048163049       （特约）商银信（商户清结算）  1
20121            410350260510021      （特约）厦门平安（商银信T1）  2
20303            423330159330022         （特约）商银信（清算款）  3
银博盛世
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
银通支付
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
银通支付
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
高汇通
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
银信联
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]


金运通
          交易对方卡号         交易对方名称  0
12942  211800293  金运通网络支付股份有限公司  4
邦付宝
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
飞银智能
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
理房通
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
畅捷通
                交易对方卡号           交易对方名称  0
41371  900113458120000  北京畅捷通支付技术有限公司备付  6
盛迪嘉
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
广东广物
Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []
传化支付
                  交易对方卡号    交易对方名称  0
12129  19000101040025807  传化支付有限公司  1
13006          220800312  传化支付有限公司  1
(37754, 3)


In [10]:
df['交易对方卡号'] = df.apply(lambda x:(x['查询卡号']+'_'+x['交易对方卡号']) if x['交易对方卡号'] in df_匹配2['交易对方卡号'].tolist() else x['交易对方卡号'],axis = 1)
print(df[df['交易对方卡号'].str.contains('215500690')])

       银行          客户名称              客户证件号码                  查询账号  \
1343        南京峰宁江建材有限公司           067085416     01690120210004417   
1427        南京峰宁江建材有限公司           067085416     01690120210004417   
54837      扬州国祥游乐设备有限公司          MA1MYGTG-3  32050174863600000366   
133568      南京荣陆顺运输有限公司  91320113MA1MF4D474   4301012119100101834   
133569      南京荣陆顺运输有限公司  91320113MA1MF4D474   4301012119100101834   
162813              王介平  321088196903083413      4340621330094149   
207792     江苏香尧建材贸易有限公司           MA1N7N019          526169863140   
224267              庞建林  321001197508230012          478054037089   
226983               张进  320123198003104851   1605020101123766308   
226996               张进  320123198003104851   1605020101123766308   
227169               张进  320123198003104851   1605020101123766308   
227182               张进  320123198003104851   1605020101123766308   
227183               张进  320123198003104851   1605020101123766308   
227184               张进  320123198

In [53]:
# df.apply(lambda x:1 if x['交易对方卡号'] in df_匹配2['交易对方卡号'].tolist() else 0,axis =1)

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
440114    1
440115    1
440116    1
440117    0
440118    0
440119    1
440120    0
440121    1
440122    1
440123    1
440124    1
440125    1
440126    1
440127    1
440128    0
440129    1
440130    0
440131    1
440132    0
440133    0
440134    0
440135    0
440136    0
440137    1
440138    1
440139    1
440140    1
440141    1
440142    1
440143    0
Length: 440144, dtype: int64

In [15]:
# 提取交易对方卡号中三方，提取内容输出到df_匹配2
df_交易对手5 = df_交易对手3[~df_交易对手3['交易对方卡号'].isin(df_匹配2['交易对方卡号'])]
df_匹配5 = pd.DataFrame()
df_筛选5 = df_交易对手2[df_交易对手2['交易对方卡号'].str.contains('^2[0-9]{1}[\*]{5}[0-9]{2}$')]
df_匹配5 = df_匹配5.append(df_筛选5)
print(df_筛选5)

Empty DataFrame
Columns: [交易对方卡号, 交易对方名称, 0]
Index: []


In [13]:
df.to_excel(r'C:\Users\zqy\Desktop\新建文件夹\厦门POC数据\火眼模板\0402\流水处理超级节点后待脱敏.xlsx',index = None)

In [14]:
df.to_csv(r'C:\Users\zqy\Desktop\新建文件夹\厦门POC数据\火眼模板\0402\流水处理超级节点后待脱敏.csv',index = None)

In [ ]:
import pandas as pd
df = pd.read_excel(r'C:\Users\zqy\Desktop\新建文件夹\厦门POC数据\火眼模板\0402\活跃程度、重要程度、中心地位.xlsx',encoding = 'gbk',sheet_name = '随机数')
import numpy as np
df_随机数 = pd.DataFrame(np.random.rand(36730,3)/100)
df['活跃程度']=df_随机数[0]
df['重要程度']=df_随机数[1]
df['中心地位']=df_随机数[2]
df['活跃程度'] = df['活跃程度'].map(lambda x:format(x, '.6f'))
df['重要程度'] = df['重要程度'].map(lambda x:format(x, '.6f'))
df['中心地位'] = df['中心地位'].map(lambda x:format(x, '.6f'))
df.to_excel(r'C:\Users\zqy\Desktop\新建文件夹\厦门POC数据\火眼模板\0402\活跃程度、重要程度、中心地补充.xlsx',encoding = 'gbk',index =None)

In [ ]:
# 用vlookup，不该用的……该用pd.merge

In [4]:
import pandas as pd

df = pd.read_excel(r'C:\Users\zqy\Desktop\新建文件夹\厦门POC数据\火眼模板\0402\流水处理超级节点后脱敏.xlsx',encoding ='gbk')

In [5]:
df

,银行,客户名称,客户证件号码,查询账号,查询卡号,交易对方证件号码,交易对方账号,交易对方卡号,交易对方名称,交易类型,...,交易渠道,备注,对公对私标志,交易对手对公对私标志,活跃程度,重要程度,中心地位,对手活跃程度,对手重要程度,对手中心地位
0,NaN,b4c494e2b57305ea16c1963272e91468建材有限公司,1d930fdc3b690ab16aa33c89cb48edb3,01600120210017464,01600120210017464,NaN,NaN,6223050311413648,陈0055e9ec6b6fd0d0b038249d3228a4da,NaN,...,NaN,还款,1,0,0.003521,0.003521,0.345079,0.006247,0.009002,0.004453
1,NaN,b4c494e2b57305ea16c1963272e91468建材有限公司,1d930fdc3b690ab16aa33c89cb48edb3,01600120210017464,01600120210017464,NaN,NaN,6223050311413648,陈0055e9ec6b6fd0d0b038249d3228a4da,NaN,...,NaN,存款,1,0,0.003521,0.003521,0.345079,0.006247,0.009002,0.004453
2,NaN,b4c494e2b57305ea16c1963272e91468建材有限公司,1d930fdc3b690ab16aa33c89cb48edb3,01600120210017464,01600120210017464,NaN,NaN,6223050311413648,陈0055e9ec6b6fd0d0b038249d3228a4da,NaN,...,NaN,还款,1,0,0.003521,0.003521,0.345079,0.006247,0.009002,0.004453
3,NaN,b4c494e2b57305ea16c1963272e91468建材有限公司,1d930fdc3b690ab16aa33c89cb48edb3,01600120210017464,01600120210017464,NaN,NaN,6223050311413648,陈0055e9ec6b6fd0d0b038249d3228a4da,NaN,...,NaN,还款,1,0,0.003521,0.003521,0.345079,0.006247,0.009002,0.004453
4,NaN,b4c494e2b57305ea16c1963272e91468建材有限公司,1d930fdc3b690ab16aa33c89cb48edb3,01600120210017464,01600120210017464,NaN,NaN,6228480391032256219,陈0055e9ec6b6fd0d0b038249d3228a4da,NaN,...,NaN,还款,1,0,0.003521,0.003521,0.345079,0.009218,0.004062,0.009280
5,NaN,b4c494e2b57305ea16c1963272e91468建材有限公司,1d930fdc3b690ab16aa33c89cb48edb3,01600120210017464,01600120210017464,NaN,NaN,6228480391032256219,陈0055e9ec6b6fd0d0b038249d3228a4da,NaN,...,NaN,还款,1,0,0.003521,0.003521,0.345079,0.009218,0.004062,0.009280
6,NaN,b4c494e2b57305ea16c1963272e91468建材有限公司,1d930fdc3b690ab16aa33c89cb48edb3,01600120210017464,01600120210017464,NaN,NaN,6223050311413648,陈0055e9ec6b6fd0d0b038249d3228a4da,NaN,...,NaN,还款,1,0,0.003521,0.003521,0.345079,0.006247,0.009002,0.004453
7,NaN,b4c494e2b57305ea16c1963272e91468建材有限公司,1d930fdc3b690ab16aa33c89cb48edb3,01600120210017464,01600120210017464,NaN,NaN,6223050311413648,陈0055e9ec6b6fd0d0b038249d3228a4da,NaN,...,NaN,还款,1,0,0.003521,0.003521,0.345079,0.006247,0.009002,0.004453
8,NaN,b4c494e2b57305ea16c1963272e91468建材有限公司,1d930fdc3b690ab16aa33c89cb48edb3,01600120210017464,01600120210017464,NaN,NaN,6223050311413648,陈0055e9ec6b6fd0d0b038249d3228a4da,NaN,...,NaN,存款,1,0,0.003521,0.003521,0.345079,0.006247,0.009002,0.004453
9,NaN,b4c494e2b57305ea16c1963272e91468建材有限公司,1d930fdc3b690ab16aa33c89cb48edb3,01600120210017464,01600120210017464,NaN,NaN,010115622300870000010,待b48c2e8318eb2af218ee3e9fc90272f5,NaN,...,NaN,存款,1,0,0.003521,0.003521,0.345079,0.008269,0.009346,0.005568


In [7]:
import re
df = df.applymap(lambda x: re.sub('nan','',str(x)))
df.to_excel(r'C:\Users\zqy\Desktop\新建文件夹\厦门POC数据\火眼模板\0402\流水处理超级节点后脱敏加SNA.xlsx',index = None)

In [10]:
# df['活跃程度'] = df['活跃程度'].map(lambda x:format(float(x), '.6f'))
# df['重要程度'] = df['重要程度'].map(lambda x:format(float(x), '.6f'))
# df['中心地位'] = df['中心地位'].map(lambda x:format(float(x), '.6f'))
df['对手活跃程度'] = df['对手活跃程度'].map(lambda x:format(float(x), '.6f'))
df['对手重要程度'] = df['对手重要程度'].map(lambda x:format(float(x), '.6f'))
df['对手中心地位'] = df['对手中心地位'].map(lambda x:format(float(x), '.6f'))
df.to_excel(r'C:\Users\zqy\Desktop\新建文件夹\厦门POC数据\火眼模板\0402\流水处理超级节点后脱敏加SNA-0402.xlsx',index = None)